<a href="https://colab.research.google.com/github/carlosinator/cil-sentiment/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports & Setup

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
%%bash
pip3 install transformers emoji==0.6.0 keras_nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 576.5/576.5 kB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.9/851.9 kB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 115.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 124.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [3]:
!git clone https://github.com/carlosinator/cil-sentiment.git

Cloning into 'cil-sentiment'...
remote: Enumerating objects: 319, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 319 (delta 117), reused 152 (delta 94), pack-reused 128
Receiving objects: 100% (319/319), 2.60 MiB | 15.67 MiB/s, done.
Resolving deltas: 100% (173/173), done.


In [4]:
import tensorflow as tf
import tensorflow_probability as tfp
import keras_nlp
import numpy as np
import pandas as pd
import sklearn
from sklearn import metrics
import transformers
from transformers import AutoTokenizer, TFAutoModel, AutoConfig, TFAutoModelForSequenceClassification
import matplotlib.pyplot as plt
import pickle
import re
import subprocess as sp
import os
from threading import Thread , Timer
import sched, time

import sys
sys.path.append("./cil-sentiment/models")
sys.path.append("./cil-sentiment/")
import utils

# reproducibility
transformers.set_seed(0) # sets the seed in random, numpy, and tf

Using TensorFlow backend


In [5]:
!gsutil cp "gs://cil_2023/train_pos_full_preprocessed_without_duplicates.txt" .
!gsutil cp "gs://cil_2023/train_neg_full_preprocessed_without_duplicates.txt" .

Copying gs://cil_2023/train_pos_full_preprocessed_without_duplicates.txt...
- [1 files][ 74.6 MiB/ 74.6 MiB]   11.5 MiB/s                                   
Operation completed over 1 objects/74.6 MiB.                                     
Copying gs://cil_2023/train_neg_full_preprocessed_without_duplicates.txt...
| [1 files][ 91.6 MiB/ 91.6 MiB]   12.3 MiB/s                                   
Operation completed over 1 objects/91.6 MiB.                                     


In [6]:
model_name = "vinai/bertweet-base"
filename_train_pos = "train_pos_full_preprocessed_without_duplicates.txt"
filename_train_neg = "train_neg_full_preprocessed_without_duplicates.txt"

# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.TPUStrategy(tpu)

In [7]:
# read dataset
dataset_pos_pd = pd.read_table(filename_train_pos, sep='\r\n', header=None, names=['text'])
dataset_neg_pd = pd.read_table(filename_train_neg, sep='\r\n', header=None, names=['text'])
dataset_pos_pd['label'] = 0
dataset_neg_pd['label'] = 1
dataset_pd = pd.concat([dataset_pos_pd, dataset_neg_pd])

# shuffle
dataset_pd = dataset_pd.sample(frac=1, random_state=0).reset_index(drop=True)

# tokenize data set
tokenizer = AutoTokenizer.from_pretrained(model_name)
texts = tokenizer.batch_encode_plus(dataset_pd['text'].tolist(),
                                    padding=True, truncation=True,
                                    return_tensors='tf')

dataset = tf.data.Dataset.from_tensor_slices((dict(texts), dataset_pd['label']))

# split training / validation
batch_size = 256 # * tpu_strategy.num_replicas_in_sync
AUTOTUNE = tf.data.experimental.AUTOTUNE

val_data_size = int(0.1 * len(dataset_pd.index))
test_data_size = int(0.1 * len(dataset_pd.index))
train_data_size = len(dataset_pd.index) - val_data_size - test_data_size
val_ds = dataset.take(val_data_size).batch(batch_size, drop_remainder=True)
test_ds = dataset.take(test_data_size).batch(batch_size, drop_remainder=True)
train_ds = dataset.skip(val_data_size).batch(batch_size, drop_remainder=True)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

<ipython-input-7-a25b9d79ca72>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataset_pos_pd = pd.read_table(filename_train_pos, sep='\r\n', header=None, names=['text'])
<ipython-input-7-a25b9d79ca72>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataset_neg_pd = pd.read_table(filename_train_neg, sep='\r\n', header=None, names=['text'])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# redefine train ds
train_ds = dataset.batch(batch_size, drop_remainder=True)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

# Train model for inference

In [9]:
import subprocess as sp
import os
from threading import Thread , Timer
import sched, time

gpu_hist = {
    "mib" : [],
    "percent" : [],
    "counter" : 0,
}

def get_gpu_memory():

    out = !nvidia-smi --query-gpu=memory.used,utilization.gpu --format=csv

    float_pattern = r'\d+\.\d+|\d+'
    numbers = re.findall(float_pattern, out[1])

    # Extract the MiB value and the percentage
    mib_val = float(numbers[0])
    perc = float(numbers[1])

    gpu_hist["counter"] += 1

    # print(mib_val, perc)

    gpu_hist["mib"].append(mib_val)
    gpu_hist["percent"].append(perc)

    return


def track_gpu_mem(interval=5.0, dictname="gpu_hist.pkl"):
    """
        This function calls itself every 5 secs and print the gpu_memory.
    """
    thd = Timer(interval, track_gpu_mem, [interval, dictname])
    thd.start()
    get_gpu_memory()

    gpu_hist["interval"] = interval

    with open(dictname, 'wb') as f:
      pickle.dump(gpu_hist, f)

    return thd

In [10]:
USE_MODEL = "read"
READ_GRU_UNITS = 64
LEARNING_RATE = 3e-4
EPOCHS = 5
model = utils.get_model(model_name, LEARNING_RATE, USE_MODEL, num_gru_units=READ_GRU_UNITS, train_data_size=train_data_size)
history = {}

run_name = "submission_inference_" + USE_MODEL + "_fullmodel_" + f"lr{LEARNING_RATE}_ep{EPOCHS}_batchs={batch_size}_gru_units={READ_GRU_UNITS}"
hd_name = run_name + "_dict.pkl"
gpu_hist_name = "gpu_hist_" + run_name + ".pkl"

Some layers from the model checkpoint at vinai/bertweet-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/bertweet-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [11]:
tf.keras.backend.clear_session()

In [12]:
callback = tf.keras.callbacks.EarlyStopping(patience=3) # early stopping def
gpu_mem_proc = track_gpu_mem(10.0, dictname=gpu_hist_name) # initialize gpu tracking

history = model.fit(
    train_ds,
    #validation_data=val_ds,
    epochs=EPOCHS,
    verbose=1,
    #callbacks=[callback]
    )

gpu_mem_proc.join()

Epoch 1/5
8868/8868 [==============================] - 1784s 197ms/step - loss: 0.2720 - sparse_categorical_accuracy: 0.8826
Epoch 2/5
8868/8868 [==============================] - 1748s 197ms/step - loss: 0.2516 - sparse_categorical_accuracy: 0.8926
Epoch 3/5
8868/8868 [==============================] - 1746s 197ms/step - loss: 0.2456 - sparse_categorical_accuracy: 0.8955
Epoch 4/5
8868/8868 [==============================] - 1746s 197ms/step - loss: 0.2417 - sparse_categorical_accuracy: 0.8972
Epoch 5/5
8868/8868 [==============================] - 1746s 197ms/step - loss: 0.2386 - sparse_categorical_accuracy: 0.8986


In [13]:
# save model
model.save(run_name)
# save training history
with open(hd_name, 'wb') as f:
  pickle.dump(history, f)

#Note: the gpu history should already been stored at gpu_hist_name

!gsutil cp -r {run_name + "/"} "gs://cil_2023/models/"
!gsutil cp {hd_name} "gs://cil_2023/models/"
!gsutil cp {gpu_hist_name} "gs://cil_2023/models/"

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


Copying file://submission_inference_read_fullmodel_lr0.0003_ep5_batchs=256_gru_units=64/keras_metadata.pb [Content-Type=application/octet-stream]...
Copying file://submission_inference_read_fullmodel_lr0.0003_ep5_batchs=256_gru_units=64/fingerprint.pb [Content-Type=application/octet-stream]...
Copying file://submission_inference_read_fullmodel_lr0.0003_ep5_batchs=256_gru_units=64/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://submission_inference_read_fullmodel_lr0.0003_ep5_batchs=256_gru_units=64/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite

# Retrieve model for inference

In [ ]:
# uncomment if trained_model is not on local machine
# !gsutil cp -r {"gs://cil_2023/models/" + run_name} .
trained_model = tf.keras.models.load_model(run_name)
trained_model.summary()

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLaye  multiple                  134309376 
 r)                                                              
                                                                 
 classifier (TFRobertaClass  multiple                  592130    
 ificationHead)                                                  
                                                                 
Total params: 134901506 (514.61 MB)
Trainable params: 134901506 (514.61 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Test accuracy

In [ ]:
preds = trained_model.predict(test_ds)

if USE_MODEL == "basemodel":
  preds = tf.keras.layers.Softmax()(preds)